# tf.function and autograph
* [source site](https://www.tensorflow.org/beta/tutorials/eager/tf_function)

In [1]:
import tensorflow as tf
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}: {}'.format(error_class, e))
    except Exception as e:
        print('Got unexpected exception \n  {}: {}'.format(type(e), e))
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))

In [6]:
assert_raises(print('d'))

d


In [14]:
# A function is like an op

@tf.function
def tf_add(a, b):
    return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: id=89, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [18]:
tf_add(3,3)

<tf.Tensor: id=100, shape=(), dtype=int32, numpy=6>

In [16]:
def add(a, b):
    return a + b
add(3,3)

6

In [21]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = tf_add(v, 3.0)
tape.gradient(result, v)

<tf.Tensor: id=162, shape=(), dtype=float32, numpy=1.0>

In [23]:
# You can use functions inside functions

@tf.function
def dense_layer(x, w, b):
    return tf_add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=209, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

### tensor function can be polymorphic.

In [26]:
# Functions are polymorphic

@tf.function
def double(a):
  print("Tracing with", a)
  return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [27]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string)) # receive only string variable

Obtaining concrete trace
Tracing with Tensor("a:0", dtype=string)


In [28]:
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))

Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


In [29]:
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1)) # data type error

Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_243 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_243]


In [59]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(tf.equal(x % 2, 0), x // 2, 3 * x + 1) # where x is even number? if is true return quotient(몫) from
                                                           # diving 2, or not reture 3 * x + 1.

print(next_collatz(tf.constant([1, 2])))


Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)


In [60]:
# We specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))

Caught expected exception 
  <class 'ValueError'>: Python inputs incompatible with input_signature: inputs ((<tf.Tensor: id=383, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>,)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name=None),))


In [47]:
tf.where(tf.equal(2 % 2, 0), 2 // 2, 3 * 2 + 1)

<tf.Tensor: id=298, shape=(), dtype=int32, numpy=1>

In [61]:
tf.where(True, 2, 7)

<tf.Tensor: id=388, shape=(), dtype=int32, numpy=2>

In [63]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)

Tracing with num_steps = 10
Tracing with num_steps = 20


In [64]:
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


### Side effects in tf.function

In [66]:
@tf.function
def f(x):
    print("Traced with", x)
    tf.print("Executed with", x)

f(1)
f(1)
f(2)

Traced with 1
Executed with 1
Executed with 1
Traced with 2
Executed with 2


In [67]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)

@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])

f(1)
f(1)
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

W0710 16:35:33.744632 140513504110336 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0710 16:35:33.746358 140513520895744 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0710 16:35:33.747229 140513504110336 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32


Python side effect
Python side effect
Python side effect


In [68]:
len(external_list) == 3

True

In [69]:
external_list[0].numpy() == 1

True

In [70]:
external_list[0]

<tf.Tensor: id=659, shape=(), dtype=int32, numpy=1>

In [71]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var:", external_var)

iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


In [79]:

x = iter([0, 1, 2, 3])
l = [0, 1, 2, 3]
for i in l:
    print(i)
    print(next(x))
   # print(next(i))

0
0
1
1
2
2
3
3


In [80]:
tf_var = tf.Variable(0)
print(tf_var)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>


In [81]:
tf_var.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=1>

In [82]:
print(tf_var)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>


### python iterator don't work with tf.function.
### so you have to use tf.data.Dataset.

In [83]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print("{}({}) contains {} nodes in its graph".format(
      f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))

@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y - x) # Some dummy computation.
    return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

train([(1, 1), (1, 1)]) contains 8 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph


In [84]:
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))

W0710 17:00:42.580288 140519420880704 deprecation.py:323] From /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph


In [89]:
test_data = [(1,2)] *2
for x, y in test_data:
    print(x)
    print(y)

1
2
1
2


In [91]:
small_tf_data = tf.data.Dataset.from_generator(lambda: small_data, (tf.int32, tf.int32))

In [92]:
measure_graph_size(train, small_tf_data)

train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph


In [94]:
for x, y in small_tf_data:
    print(x)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)


In [95]:
for x, y in test_data:
    print(x)

1
1


## Automatic control dependencies

In [115]:
# Automatic control dependencies

a = tf.Variable(4.0)
b = tf.Variable(6.0)


@tf.function
def f(x, y):
    a.assign(y * b) # 2.0 x 6.0 = 12.0 -> a
    b.assign_add(x * a) # 1.0 x 12.0 + 6.0 = 18.0
    return a + b 

f(1.0, 2.0)  # 30.0

<tf.Tensor: id=1539, shape=(), dtype=float32, numpy=30.0>

In [133]:
v = None

print(v)
@tf.function
def f(x):
    if v is None:
         v = tf.Variable(1.0)
    print(v)
    return v.assign_add(x)

with assert_raises(ValueError):
    f(1.0)

None
Got unexpected exception 
  <class 'AttributeError'>: in converted code:

    <ipython-input-133-cd10815703a4>:9 f  *
        return v.assign_add(x)
    /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:329 converted_call
        f = getattr(owner, f)

    AttributeError: 'Undefined' object has no attribute 'assign_add'



In [121]:
# Non-ambiguous code is ok though

v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(1.0))  # 2.0
print(f(2.0))  # already v is converted 2.0, so result will be 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [122]:
print(f(1.0))  # from v=4.0
print(f(2.0))  # from v=5.0

tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(7.0, shape=(), dtype=float32)


In [132]:
# You can also create variables inside a tf.function as long as we can prove
# that those variables are created only the first time the function is executed.

class C: pass
obj = C(); obj.v = None

@tf.function
def g(x):
    if obj.v is None:
        obj.v = tf.Variable(1.0)
    print(obj.v)
    return obj.v.assign_add(x)

print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

<tf.Variable 'Variable:0' shape=() dtype=float32>
<tf.Variable 'Variable:0' shape=() dtype=float32>
tf.Tensor(2.0, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=() dtype=float32>
tf.Tensor(4.0, shape=(), dtype=float32)


In [136]:
# Variable initializers can depend on function arguments and on values of other
# variables. We can figure out the right initialization order using the same
# method we use to generate control dependencies.

state = []
@tf.function
def fn(x):
    if not state:
        state.append(tf.Variable(2.0 * x)) # 1.0 -> state[0] = 2.0
        state.append(tf.Variable(state[0] * 3.0)) # state[1] = 6.0
    return state[0] * x * state[1] #  (2.0 * x * 6.0)

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0))) # state has values, so doesn't append any values.
state

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>]

In [152]:
# Simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        tf.print(tf.reduce_sum(x))
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))


[0.953889966 0.30712533 0.919335485 0.0907039642 0.898069382]
3.16912413
[0.741539061 0.297819644 0.725582898 0.0904560387 0.715356529]
2.57075429
[0.630074143 0.289316 0.620355606 0.0902101323 0.614024818]
2.24398065
[0.558103263 0.281505138 0.551375628 0.0899662226 0.546954036]
2.02790451
[0.506568789 0.274297535 0.501550496 0.0897242799 0.498233855]
1.87037492
[0.467267454 0.267619133 0.463335663 0.0894842818 0.460727036]
1.74843359
[0.435988843 0.261408031 0.432798982 0.0892462 0.430676609]
1.65011871
[0.410314113 0.255612046 0.407657832 0.0890100077 0.40588662]
1.56848061
[0.388739377 0.250186771 0.386482179 0.0887756869 0.38497448]
1.4991585
[0.370272934 0.245094225 0.368323594 0.0885432065 0.367019713]
1.43925357
[0.354230404 0.240301654 0.352524489 0.0883125439 0.351382136]
1.38675129
[0.340121925 0.235780656 0.338612467 0.088083677 0.337600708]
1.34019947
[0.327586234 0.231506467 0.326238096 0.0878565758 0.325333714]
1.29852104
[0.316350222 0.227457374 0.315136492 0.0876312256

<tf.Tensor: id=2705, shape=(5,), dtype=float32, numpy=
array([0.23986073, 0.19354892, 0.23933268, 0.08526165, 0.23897672],
      dtype=float32)>

In [150]:
# If you're curious you can inspect the code autograph generates.
# It feels like reading assembly language, though.
# note that it omits @tf.function

def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    tf.print(tf.reduce_sum(x))
    x = tf.tanh(x)
  return x

print(tf.autograph.to_code(f))

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()

  def loop_test(x_1):
    return ag__.converted_call('reduce_sum', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None) > 1

  def loop_body(x_1):
    ag__.converted_call('print', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)
    ag__.converted_call('print', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (ag__.converted_call('reduce_sum', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None),), None)
    x_1 = ag__.converted_call('tanh', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)
    return x

### Autograph and condition

In [153]:
def test_tf_cond(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'cond' for node in g.as_graph_def().node):
        print("{}({}) uses tf.cond.".format(
            f.__name__, ', '.join(map(str, args))))
    else:
        print("{}({}) executes normally.".format(
          f.__name__, ', '.join(map(str, args))))

ConversionError: converting <tensorflow.python.eager.def_function.Function object at 0x7fcc6843dbe0>: AttributeError: 'Function' object has no attribute '__code__'